# Notebook 2: Análisis de Nulos y Limpieza de Datos
## Monitor de Vulnerabilidad Económica - Colombia

**Objetivo:** Analizar valores nulos en el dataset IPM para identificar casos de "No Aplica" según la estructura modular de la encuesta GEIH.

**Dataset:** `data/processed/geih_2024_ipm_variables.csv`

---

## Contexto de la Encuesta GEIH

La GEIH tiene una **estructura modular** donde ciertas preguntas solo aplican a subgrupos específicos:

### Módulos y su aplicabilidad:

1. **Características Generales**: Todas las personas
2. **Datos del Hogar**: Solo jefe del hogar o persona informante
3. **Fuerza de Trabajo (FT)**: Solo personas ≥ 10 años
4. **Ocupados**: Solo personas ocupadas (que trabajan)
5. **No Ocupados**: Solo desocupados o inactivos
6. **Niñez**: Solo niños < 18 años
7. **Servicios**: Variables del hogar (no persona)

**Por tanto, muchos "nulos" son en realidad "NO APLICA"** según el módulo y características de la persona.

---


In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Librerías importadas")


✓ Librerías importadas


In [3]:
# Cargar dataset IPM
df = pd.read_csv('../data/processed/geih_2024_ipm_variables.csv')

print(f"Dataset cargado:")
print(f"  Registros: {df.shape[0]:,}")
print(f"  Columnas: {df.shape[1]}")
print(f"\nPrimeras filas:")
df.head(3)


Dataset cargado:
  Registros: 688,507
  Columnas: 37

Primeras filas:


,P6040,P5110,P6110,P5020,P6585S2,P6016,P6120,P6585S3,P6080,P6050,DIRECTORIO,P6090,P6430,INGLABO,PERIODO,FT,P6920,P5100,P5000,DPTO,P5080,P5040,HOGAR,CLASE,AREA,P5090,P6250,P6240,P5030,P6585S1,P6160,P6100,ORDEN,P6170,P5070,P5010,SECUENCIA_P
0,39,NaN,9.0,1,NaN,1,NaN,NaN,6,1,7655976,1,NaN,NaN,20240101,NaN,NaN,NaN,2,5,3.0,1,1,1,5.0,3,2.0,4.0,1.0,NaN,1.0,1.0,1,2.0,1,1,1
1,32,NaN,1.0,1,1.0,2,46500.0,1.0,6,2,7655976,1,1.0,1160000.0,20240101,1.0,1.0,NaN,2,5,3.0,1,1,1,5.0,3,NaN,1.0,1.0,2.0,1.0,1.0,2,2.0,1,1,1
2,3,NaN,5.0,1,NaN,3,NaN,NaN,6,3,7655976,1,NaN,NaN,20240101,NaN,NaN,NaN,2,5,3.0,1,1,1,5.0,3,NaN,NaN,1.0,NaN,2.0,1.0,3,2.0,1,1,1


In [4]:
# Análisis de valores nulos
null_summary = pd.DataFrame({
    'Variable': df.columns,
    'Nulos': df.isnull().sum(),
    '% Nulos': (df.isnull().sum() / len(df) * 100).round(2),
    'No_Nulos': df.notnull().sum()
}).sort_values('% Nulos', ascending=False)

print("Variables con valores nulos:\n")
print(null_summary[null_summary['Nulos'] > 0])


Variables con valores nulos:

        Variable   Nulos  % Nulos  No_Nulos
P5100      P5100  663816    96.41     24691
P6585S3  P6585S3  532604    77.36    155903
P6585S1  P6585S1  532604    77.36    155903
P6585S2  P6585S2  532604    77.36    155903
P6120      P6120  516476    75.01    172031
P5110      P5110  446329    64.83    242178
P6250      P6250  417145    60.59    271362
INGLABO  INGLABO  406793    59.08    281714
P6430      P6430  394214    57.26    294293
P6920      P6920  394214    57.26    294293
P6110      P6110  387721    56.31    300786
FT            FT  355395    51.62    333112
AREA        AREA  192278    27.93    496229
P6240      P6240  142463    20.69    546044
P6100      P6100   26573     3.86    661934
P6160      P6160   21482     3.12    667025
P6170      P6170   21482     3.12    667025
P5030      P5030   12909     1.87    675598
P5080      P5080    4560     0.66    683947


## Análisis por Variable según Módulo GEIH

Ahora vamos a analizar cada variable con nulos para determinar si son "No Aplica" legítimos:

### Variables con >50% nulos a analizar:

1. **P5100 (Gas) - 96.41%**: Variable del HOGAR, solo jefe/informante
2. **P6585S1/S2/S3 (Niñez) - 77.36%**: Solo niños < 18 años
3. **P6120 (Cuidado niños) - 75.01%**: Solo hogares con niños < 5 años
4. **P5110 (Energía) - 64.83%**: Variable del HOGAR
5. **P6250 (Tipo empleo) - 60.59%**: Solo OCUPADOS
6. **INGLABO (Ingreso laboral) - 59.08%**: Solo OCUPADOS
7. **P6430, P6920 (Trabajo) - 57.26%**: Solo OCUPADOS
8. **P6110 (Mortalidad) - 56.31%**: Solo mujeres en edad fértil
9. **FT (Fuerza trabajo) - 51.62%**: Solo personas ≥ 10 años

---


In [5]:
# 1. Analizar FT (Fuerza de Trabajo) - Solo aplica a personas >= 10 años
# P6040 es la edad

print("="*60)
print("ANÁLISIS 1: FT (Fuerza de Trabajo)")
print("="*60)

# Crear indicador de edad >= 10
df['aplica_ft'] = df['P6040'] >= 10

print(f"\nPersonas >= 10 años: {df['aplica_ft'].sum():,} ({df['aplica_ft'].sum()/len(df)*100:.1f}%)")
print(f"Personas < 10 años: {(~df['aplica_ft']).sum():,} ({(~df['aplica_ft']).sum()/len(df)*100:.1f}%)")

# Comparar con nulos de FT
print(f"\nNulos en FT: {df['FT'].isnull().sum():,} ({df['FT'].isnull().sum()/len(df)*100:.1f}%)")
print(f"FT con valor: {df['FT'].notnull().sum():,} ({df['FT'].notnull().sum()/len(df)*100:.1f}%)")

# Verificar si los nulos corresponden a < 10 años
nulos_ft_menores = df[df['FT'].isnull() & (~df['aplica_ft'])].shape[0]
nulos_ft_mayores = df[df['FT'].isnull() & df['aplica_ft']].shape[0]

print(f"\nNulos FT en < 10 años (NO APLICA): {nulos_ft_menores:,}")
print(f"Nulos FT en >= 10 años (REAL): {nulos_ft_mayores:,}")

print(f"\n✓ Conclusión: {nulos_ft_menores/df['FT'].isnull().sum()*100:.1f}% de nulos en FT son NO APLICA")


ANÁLISIS 1: FT (Fuerza de Trabajo)

Personas >= 10 años: 599,834 (87.1%)
Personas < 10 años: 88,673 (12.9%)

Nulos en FT: 355,395 (51.6%)
FT con valor: 333,112 (48.4%)

Nulos FT en < 10 años (NO APLICA): 88,673
Nulos FT en >= 10 años (REAL): 266,722

✓ Conclusión: 25.0% de nulos en FT son NO APLICA


In [6]:
# 2. Analizar variables de OCUPADOS (INGLABO, P6250, P6430, P6920)
# Estas solo aplican a personas ocupadas (FT == 1)

print("\n" + "="*60)
print("ANÁLISIS 2: Variables de OCUPADOS")
print("="*60)

# FT == 1 significa ocupado
df['es_ocupado'] = df['FT'] == 1

print(f"\nPersonas ocupadas (FT=1): {df['es_ocupado'].sum():,} ({df['es_ocupado'].sum()/len(df)*100:.1f}%)")
print(f"Personas NO ocupadas: {(~df['es_ocupado']).sum():,} ({(~df['es_ocupado']).sum()/len(df)*100:.1f}%)")

# Analizar cada variable
variables_ocupados = ['INGLABO', 'P6250', 'P6430', 'P6920']

for var in variables_ocupados:
    if var in df.columns:
        nulos_total = df[var].isnull().sum()
        nulos_no_ocupados = df[df[var].isnull() & (~df['es_ocupado'])].shape[0]
        nulos_ocupados = df[df[var].isnull() & df['es_ocupado']].shape[0]
        
        print(f"\n{var}:")
        print(f"  Nulos totales: {nulos_total:,} ({nulos_total/len(df)*100:.1f}%)")
        print(f"  Nulos en NO ocupados (NO APLICA): {nulos_no_ocupados:,}")
        print(f"  Nulos en ocupados (REAL): {nulos_ocupados:,}")
        
        if nulos_total > 0:
            print(f"  → {nulos_no_ocupados/nulos_total*100:.1f}% son NO APLICA")



ANÁLISIS 2: Variables de OCUPADOS

Personas ocupadas (FT=1): 333,112 (48.4%)
Personas NO ocupadas: 355,395 (51.6%)

INGLABO:
  Nulos totales: 406,793 (59.1%)
  Nulos en NO ocupados (NO APLICA): 355,395
  Nulos en ocupados (REAL): 51,398
  → 87.4% son NO APLICA

P6250:
  Nulos totales: 417,145 (60.6%)
  Nulos en NO ocupados (NO APLICA): 156,170
  Nulos en ocupados (REAL): 260,975
  → 37.4% son NO APLICA

P6430:
  Nulos totales: 394,214 (57.3%)
  Nulos en NO ocupados (NO APLICA): 355,395
  Nulos en ocupados (REAL): 38,819
  → 90.2% son NO APLICA

P6920:
  Nulos totales: 394,214 (57.3%)
  Nulos en NO ocupados (NO APLICA): 355,395
  Nulos en ocupados (REAL): 38,819
  → 90.2% son NO APLICA


In [7]:
# 3. Analizar variables de NIÑEZ (P6585S1, P6585S2, P6585S3)
# Estas solo aplican a niños < 18 años

print("\n" + "="*60)
print("ANÁLISIS 3: Variables de NIÑEZ")
print("="*60)

# Crear indicador de menores de 18
df['es_menor_18'] = df['P6040'] < 18

print(f"\nMenores de 18 años: {df['es_menor_18'].sum():,} ({df['es_menor_18'].sum()/len(df)*100:.1f}%)")
print(f"Mayores o igual a 18: {(~df['es_menor_18']).sum():,} ({(~df['es_menor_18']).sum()/len(df)*100:.1f}%)")

# Analizar variables de niñez
variables_ninez = ['P6585S1', 'P6585S2', 'P6585S3']

for var in variables_ninez:
    if var in df.columns:
        nulos_total = df[var].isnull().sum()
        nulos_adultos = df[df[var].isnull() & (~df['es_menor_18'])].shape[0]
        nulos_menores = df[df[var].isnull() & df['es_menor_18']].shape[0]
        
        print(f"\n{var}:")
        print(f"  Nulos totales: {nulos_total:,} ({nulos_total/len(df)*100:.1f}%)")
        print(f"  Nulos en >= 18 años (NO APLICA): {nulos_adultos:,}")
        print(f"  Nulos en < 18 años (REAL): {nulos_menores:,}")
        
        if nulos_total > 0:
            print(f"  → {nulos_adultos/nulos_total*100:.1f}% son NO APLICA")



ANÁLISIS 3: Variables de NIÑEZ

Menores de 18 años: 176,461 (25.6%)
Mayores o igual a 18: 512,046 (74.4%)

P6585S1:
  Nulos totales: 532,604 (77.4%)
  Nulos en >= 18 años (NO APLICA): 357,038
  Nulos en < 18 años (REAL): 175,566
  → 67.0% son NO APLICA

P6585S2:
  Nulos totales: 532,604 (77.4%)
  Nulos en >= 18 años (NO APLICA): 357,038
  Nulos en < 18 años (REAL): 175,566
  → 67.0% son NO APLICA

P6585S3:
  Nulos totales: 532,604 (77.4%)
  Nulos en >= 18 años (NO APLICA): 357,038
  Nulos en < 18 años (REAL): 175,566
  → 67.0% son NO APLICA


In [8]:
# 4. Analizar P6120 (Cuidado niños menores 5 años)
# Solo aplica si hay niños menores de 5 en el hogar o si la persona es menor de 5

print("\n" + "="*60)
print("ANÁLISIS 4: P6120 (Cuidado niños < 5 años)")
print("="*60)

# Crear indicador de menores de 5
df['es_menor_5'] = df['P6040'] < 5

print(f"\nMenores de 5 años: {df['es_menor_5'].sum():,} ({df['es_menor_5'].sum()/len(df)*100:.1f}%)")
print(f"Mayores o igual a 5: {(~df['es_menor_5']).sum():,} ({(~df['es_menor_5']).sum()/len(df)*100:.1f}%)")

if 'P6120' in df.columns:
    nulos_total = df['P6120'].isnull().sum()
    con_valor = df['P6120'].notnull().sum()
    
    print(f"\nP6120:")
    print(f"  Nulos totales: {nulos_total:,} ({nulos_total/len(df)*100:.1f}%)")
    print(f"  Con valor: {con_valor:,} ({con_valor/len(df)*100:.1f}%)")
    
    # Esta variable puede aplicar a varios miembros del hogar si hay niños < 5
    # Por ahora, identificamos menores de 5
    nulos_mayores_5 = df[df['P6120'].isnull() & (~df['es_menor_5'])].shape[0]
    print(f"  Nulos en >= 5 años: {nulos_mayores_5:,}")
    print(f"  → Muchos de estos pueden ser NO APLICA (sin niños en el hogar)")



ANÁLISIS 4: P6120 (Cuidado niños < 5 años)

Menores de 5 años: 38,847 (5.6%)
Mayores o igual a 5: 649,660 (94.4%)

P6120:
  Nulos totales: 516,476 (75.0%)
  Con valor: 172,031 (25.0%)
  Nulos en >= 5 años: 477,638
  → Muchos de estos pueden ser NO APLICA (sin niños en el hogar)


In [9]:
# 5. Analizar variables de SERVICIOS (P5100, P5110, etc.)
# Variables de hogar que pueden tener nulos por AREA rural

print("\n" + "="*60)
print("ANÁLISIS 5: Variables de SERVICIOS")
print("="*60)

# AREA: 1=Urbano, 2=Rural
if 'AREA' in df.columns:
    print(f"\nDistribución AREA:")
    print(df['AREA'].value_counts().sort_index())
    
    # Analizar servicios por área
    variables_servicios = ['P5070', 'P5080', 'P5090', 'P5100', 'P5110']
    
    for var in variables_servicios:
        if var in df.columns:
            nulos_total = df[var].isnull().sum()
            
            # Nulos por área
            nulos_urbano = df[(df[var].isnull()) & (df['AREA'] == 1)].shape[0]
            nulos_rural = df[(df[var].isnull()) & (df['AREA'] == 2)].shape[0]
            nulos_sin_area = df[(df[var].isnull()) & (df['AREA'].isnull())].shape[0]
            
            print(f"\n{var}:")
            print(f"  Nulos totales: {nulos_total:,} ({nulos_total/len(df)*100:.1f}%)")
            print(f"  Nulos en urbano: {nulos_urbano:,}")
            print(f"  Nulos en rural: {nulos_rural:,}")
            print(f"  Nulos sin AREA definida: {nulos_sin_area:,}")
            
            # Los nulos sin AREA son problemáticos
            if nulos_sin_area > 0:
                print(f"  ⚠ {nulos_sin_area:,} nulos sin AREA definida (problema de merge)")



ANÁLISIS 5: Variables de SERVICIOS

Distribución AREA:
AREA
5.0     26445
8.0     22830
11.0    22114
13.0    23768
15.0    16714
17.0    24552
18.0    16969
19.0    17359
20.0    18391
23.0    16241
27.0    20170
41.0    17401
44.0    17118
47.0    20020
50.0    18638
52.0    19087
54.0    21649
63.0    18297
66.0    23990
68.0    20887
70.0    18654
73.0    14530
76.0    23494
81.0     4100
85.0     4273
86.0     3264
88.0     7054
91.0     3568
94.0     4180
95.0     3447
97.0     4287
99.0     2738
Name: count, dtype: int64

P5070:
  Nulos totales: 0 (0.0%)
  Nulos en urbano: 0
  Nulos en rural: 0
  Nulos sin AREA definida: 0

P5080:
  Nulos totales: 4,560 (0.7%)
  Nulos en urbano: 0
  Nulos en rural: 0
  Nulos sin AREA definida: 1,672
  ⚠ 1,672 nulos sin AREA definida (problema de merge)

P5090:
  Nulos totales: 0 (0.0%)
  Nulos en urbano: 0
  Nulos en rural: 0
  Nulos sin AREA definida: 0

P5100:
  Nulos totales: 663,816 (96.4%)
  Nulos en urbano: 0
  Nulos en rural: 0
  Nulos s

## Estrategia de Limpieza de Datos

Basado en el análisis, definimos la estrategia para cada tipo de nulo:

### 1. **Nulos "NO APLICA" según módulo** → Imputar con código especial
- **FT** (< 10 años): Imputar con `-1` o crear variable indicadora
- **Variables OCUPADOS** (no ocupados): Imputar con `-1` 
- **Variables NIÑEZ** (>= 18 años): Imputar con `-1`
- **P6120** (sin niños < 5): Imputar con `-1`

### 2. **Nulos en SERVICIOS** (problema de merge con hogar) → Imputar con moda o -1
- Verificar si son por falta de datos del hogar
- Imputar con valor más frecuente o -1

### 3. **Nulos REALES** (missing data) → Análisis caso por caso
- **P6240** (20.69% nulos): Analizar patrón
- **P6100** (3.86% nulos): Poca proporción, puede eliminar o imputar
- **AREA** (27.93% nulos): Problema serio de merge

### 4. **Variables con AREA nula** → Requieren investigación
- Son personas sin datos del hogar (problema de consolidación)
- Pueden eliminarse si no son recuperables

---


In [10]:
# Investigar registros con AREA nula (problema de merge)
print("="*60)
print("ANÁLISIS CRÍTICO: Registros sin AREA (problema de merge)")
print("="*60)

sin_area = df[df['AREA'].isnull()]
print(f"\nRegistros sin AREA: {len(sin_area):,} ({len(sin_area)/len(df)*100:.1f}%)")

if len(sin_area) > 0:
    print("\nEstadísticas de registros sin AREA:")
    print(f"  Edad promedio: {sin_area['P6040'].mean():.1f} años")
    print(f"  Rango edad: {sin_area['P6040'].min():.0f} - {sin_area['P6040'].max():.0f}")
    
    # Ver parentesco
    if 'P6050' in sin_area.columns:
        print(f"\nDistribución de parentesco (P6050):")
        print(sin_area['P6050'].value_counts().head())
    
    # Ver si tienen otras variables del hogar nulas
    vars_hogar = ['P5000', 'P5010', 'P5020', 'P5030', 'P5040', 'P5070', 'P5080', 'P5090', 'P5100', 'P5110']
    vars_hogar_existentes = [v for v in vars_hogar if v in df.columns]
    
    if vars_hogar_existentes:
        nulos_hogar = sin_area[vars_hogar_existentes].isnull().sum()
        print(f"\nVariables del hogar también nulas:")
        print(nulos_hogar[nulos_hogar > 0])


ANÁLISIS CRÍTICO: Registros sin AREA (problema de merge)

Registros sin AREA: 192,278 (27.9%)

Estadísticas de registros sin AREA:
  Edad promedio: 35.4 años
  Rango edad: 0 - 108

Distribución de parentesco (P6050):
P6050
1    67677
3    65647
2    34446
8    11592
9     3436
Name: count, dtype: int64

Variables del hogar también nulas:
P5030     11449
P5080      1672
P5100    187899
P5110    123530
dtype: int64


In [11]:
# Decisión: Eliminar registros sin AREA ya que no tienen datos del hogar
# Estos registros no son útiles para el análisis IPM

print("\n" + "="*60)
print("DECISIÓN: Eliminación de registros sin datos del hogar")
print("="*60)

print(f"\nRegistros originales: {len(df):,}")

# Eliminar registros sin AREA (sin datos del hogar)
df_clean = df[df['AREA'].notnull()].copy()

print(f"Registros después de eliminar sin AREA: {len(df_clean):,}")
print(f"Registros eliminados: {len(df) - len(df_clean):,} ({(len(df) - len(df_clean))/len(df)*100:.1f}%)")

print(f"\n✓ Dataset limpio tiene {len(df_clean):,} registros válidos")



DECISIÓN: Eliminación de registros sin datos del hogar

Registros originales: 688,507
Registros después de eliminar sin AREA: 496,229
Registros eliminados: 192,278 (27.9%)

✓ Dataset limpio tiene 496,229 registros válidos


In [12]:
# Imputar nulos "NO APLICA" con -1
print("\n" + "="*60)
print("IMPUTACIÓN DE NULOS 'NO APLICA'")
print("="*60)

# 1. FT para menores de 10 años
if 'FT' in df_clean.columns:
    antes = df_clean['FT'].isnull().sum()
    df_clean.loc[df_clean['P6040'] < 10, 'FT'] = df_clean.loc[df_clean['P6040'] < 10, 'FT'].fillna(-1)
    despues = df_clean['FT'].isnull().sum()
    print(f"FT: Imputados {antes - despues:,} nulos en < 10 años con -1")

# 2. Variables de OCUPADOS para no ocupados
vars_ocupados = ['INGLABO', 'P6250', 'P6430', 'P6920']
for var in vars_ocupados:
    if var in df_clean.columns:
        antes = df_clean[var].isnull().sum()
        # Imputar -1 donde FT != 1 (no ocupados)
        mask_no_ocupado = (df_clean['FT'] != 1) | (df_clean['FT'].isnull())
        df_clean.loc[mask_no_ocupado, var] = df_clean.loc[mask_no_ocupado, var].fillna(-1)
        despues = df_clean[var].isnull().sum()
        print(f"{var}: Imputados {antes - despues:,} nulos en NO OCUPADOS con -1")

# 3. Variables de NIÑEZ para >= 18 años
vars_ninez = ['P6585S1', 'P6585S2', 'P6585S3']
for var in vars_ninez:
    if var in df_clean.columns:
        antes = df_clean[var].isnull().sum()
        df_clean.loc[df_clean['P6040'] >= 18, var] = df_clean.loc[df_clean['P6040'] >= 18, var].fillna(-1)
        despues = df_clean[var].isnull().sum()
        print(f"{var}: Imputados {antes - despues:,} nulos en >= 18 años con -1")

# 4. P6120 para personas sin relevancia (mayoría son NO APLICA)
if 'P6120' in df_clean.columns:
    antes = df_clean['P6120'].isnull().sum()
    # Imputar -1 en nulos (mayoría son NO APLICA según análisis)
    df_clean['P6120'] = df_clean['P6120'].fillna(-1)
    despues = df_clean['P6120'].isnull().sum()
    print(f"P6120: Imputados {antes - despues:,} nulos con -1 (NO APLICA)")



IMPUTACIÓN DE NULOS 'NO APLICA'
FT: Imputados 60,478 nulos en < 10 años con -1
INGLABO: Imputados 250,454 nulos en NO OCUPADOS con -1
P6250: Imputados 106,772 nulos en NO OCUPADOS con -1
P6430: Imputados 250,454 nulos en NO OCUPADOS con -1
P6920: Imputados 250,454 nulos en NO OCUPADOS con -1
P6585S1: Imputados 256,435 nulos en >= 18 años con -1
P6585S2: Imputados 256,435 nulos en >= 18 años con -1
P6585S3: Imputados 256,435 nulos en >= 18 años con -1
P6120: Imputados 352,603 nulos con -1 (NO APLICA)


In [13]:
# Imputar nulos restantes en variables de servicios con valor más frecuente
print("\n" + "="*60)
print("IMPUTACIÓN DE SERVICIOS")
print("="*60)

vars_servicios = ['P5070', 'P5080', 'P5090', 'P5100', 'P5110']

for var in vars_servicios:
    if var in df_clean.columns:
        nulos_antes = df_clean[var].isnull().sum()
        if nulos_antes > 0:
            # Imputar con la moda (valor más frecuente)
            moda = df_clean[var].mode()[0] if len(df_clean[var].mode()) > 0 else 2
            df_clean[var] = df_clean[var].fillna(moda)
            print(f"{var}: Imputados {nulos_antes:,} nulos con moda = {moda}")



IMPUTACIÓN DE SERVICIOS
P5080: Imputados 2,888 nulos con moda = 3.0
P5100: Imputados 475,917 nulos con moda = 500000.0
P5110: Imputados 322,799 nulos con moda = 98.0


In [14]:
# Verificar nulos restantes
print("\n" + "="*60)
print("VERIFICACIÓN FINAL DE NULOS")
print("="*60)

nulos_finales = df_clean.isnull().sum()
nulos_restantes = nulos_finales[nulos_finales > 0].sort_values(ascending=False)

print(f"\nVariables con nulos restantes: {len(nulos_restantes)}")
if len(nulos_restantes) > 0:
    print("\nNulos restantes por variable:")
    for var, count in nulos_restantes.items():
        pct = (count / len(df_clean) * 100)
        print(f"  {var}: {count:,} ({pct:.2f}%)")
else:
    print("\n✓ No hay nulos restantes en el dataset")



VERIFICACIÓN FINAL DE NULOS

Variables con nulos restantes: 14

Nulos restantes por variable:
  P6110: 247,113 (49.80%)
  P6250: 193,535 (39.00%)
  FT: 189,976 (38.28%)
  P6585S2: 121,267 (24.44%)
  P6585S1: 121,267 (24.44%)
  P6585S3: 121,267 (24.44%)
  P6240: 97,784 (19.71%)
  INGLABO: 38,662 (7.79%)
  P6430: 29,864 (6.02%)
  P6920: 29,864 (6.02%)
  P6100: 18,561 (3.74%)
  P6160: 14,345 (2.89%)
  P6170: 14,345 (2.89%)
  P5030: 1,460 (0.29%)


In [15]:
# Eliminar columnas auxiliares creadas para el análisis
columns_to_drop = ['aplica_ft', 'es_ocupado', 'es_menor_18', 'es_menor_5']
df_clean = df_clean.drop(columns=[col for col in columns_to_drop if col in df_clean.columns])

print(f"\n✓ Columnas auxiliares eliminadas")
print(f"Shape final: {df_clean.shape}")



✓ Columnas auxiliares eliminadas
Shape final: (496229, 37)
